# YOLO Training Tutorial

In this notebook, I will talk about training process using VOC 2012 dataset. 

### Requirements

1. Able to detect image using pretrained darknet model
2. Many Gigabytes of Disk Space
3. High Speed Internet Connection Preferred
4. GPU Preferred

### 1. Download Dataset

You can read the full description of VOC dataset [here](http://host.robots.ox.ac.uk/pascal/VOC/)

#### In command, 

먼저, VOC 데이터셋 공식 사이트에서 훈련/검증 데이터셋을 data/ 에 다운로드 받습니다. 

> wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar -O ./data/voc2012_raw.tar

만약 Windows 운영체제를 사용한다면, wget 설치 파일을 시스템 폴더에 포함시켜서 wget 명령어를 사용하거나, 사이트에서 직접 다운로드합니다. 

윈도우에서 wget 명령어를 사용하는 방법은 [이 블로그](https://sound10000w.tistory.com/229)를 참조하세요. 

다운로드 받은 VOC 2012 데이터셋은 tar 파일로 압축되어 있습니다. 압축을 푼 데이터들을 저장할 디렉터리를 하나 만들어줍니다. 

> mkdir -p ./data/voc2012_raw

tar 파일을 앞서 만든 디렉터리에 압축 해제합니다. 

> tar -xf ./data/voc2012_raw.tar -C ./data/voc2012_raw

만약 Windows 운영체제를 사용한다면, tar 압축/압축 해제 용 프로그램을 설치해야 합니다. 

설치 및 압축/압축해제 과정은 [이 블로그](https://www.itopening.com/708/)를 참조하세요. 

데이터셋을 한 번 보겠습니다. 

> ls ./data/voc2012_raw/VOCdevkit/VOC2012 

파이썬 코드로는 아래와 같습니다. 

In [2]:
import os

path = './data/voc2012_raw/VOCdevkit/VOC2012'
file_list = os.listdir(path)

print(*file_list, sep='\n')

Annotations
ImageSets
JPEGImages
SegmentationClass
SegmentationObject


### 2. Transform Dataset

See tools/voc2012.py for implementation, this format is based on [tensorflow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). 

Many fields are not required, I left them there for compatibility with official API.

#### In command, 

> python tools/voc2012.py \
  --data_dir ./data/voc2012_raw/VOCdevkit/VOC2012 \
  --split train \
  --output_file ./data/voc2012_train.tfrecord
  
> python tools/voc2012.py \
  --data_dir ./data/voc2012_raw/VOCdevkit/VOC2012 \
  --split val \
  --output_file ./data/voc2012_val.tfrecord

아래 명령을 통해 데이터셋을 visualize할 수 있습니다. 

> python tools/visualize_dataset.py --classes ./data/voc2012.names

위 명령을 커맨드에서 실행하면 데이터셋 중 하나의 이미지를 랜덤으로 output.jpg로 저장합니다. 

### 3. Training

You can adjust the parameters based on your setup

> python train.py \
	--dataset ./data/voc2012_train.tfrecord \
	--val_dataset ./data/voc2012_val.tfrecord \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--mode fit 
    --transfer darknet \
	--batch_size 16 \
	--epochs 10 \
	--weights ./checkpoints/yolov3.tf \
	--weights_num_classes 80 

python train.py \
    --dataset E:/Capstone_largefiles/DeepLearning/YOLOV3/data/koreanfood_train.tfrecord \
    --val_dataset E:/Capstone_largefiles/DeepLearning/YOLOV3/data/koreanfood_val.tfrecord \
    --weights E:/Capstone_largefiles/DeepLearning/YOLOV3/checkpoints/yolov3.tf \
    --classes E:/Capstone_largefiles/DeepLearning/YOLOV3/data/images/koreanfood/koreanfood.names \
    --num_classes 400 \
    --mode fit \
    --transfer darknet \
    --epochs 2 \
    --batch_size 8 \
    --weights_num_classes 80

I have tested this works 100% with correct loss and converging over time.
Each epoch takes around 10 minutes on single AWS p2.xlarge (Nvidia K80 GPU) Instance.

You might see warnings or error messages during training, they are not critical dont' worry too much about them.
There might be a long wait time between each epoch becaues we are calculating validation loss.

#### 4. Inference

**detect from images**

> python detect.py \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--weights ./checkpoints/yolov3_train_5.tf \
	--image ./data/street.jpg

**detect from validation set**

> python detect.py \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--weights ./checkpoints/yolov3_train_5.tf \
	--tfrecord ./data/voc2012_val.tfrecord

You should see some detect objects in the standard output and the visualization at `output.jpg`.
this is just a proof of concept, so it won't be as good as pretrained models.
In my experience, you might need lower score score thershold if you didn't train it enough.